# Temperature Data Aggregation programmatically
#### Here we are going to aggregate temperature data programmatically to provide average temperature in Poland on span of 5 years, monthly 

This notebook exists because data from national database of Poland contains monthly data from every meteorological station in Poland, it contains not only temperature but also, other meteorological data 

## Reading Data
First lets read raw data from national database of Poland

The provided code snippet below is focused on data preprocessing and filtering, specifically involving the handling of CSV data using the `pandas` library in Python. It performs several key steps to prepare a dataset for analysis:

1. **Importing the pandas library**: 
   `import pandas as pd`
   This step imports the `pandas` library, which is widely used in data science for data manipulation and analysis.

2. **Loading a CSV file**: 
   `full_data = pd.read_csv("../TemperatureData/PartialYearsFiles/k_m_t_2019.csv", encoding="ISO-8859-1", header=None)`
   This line reads a CSV file into a DataFrame, a tabular data structure in pandas. The file is assumed to be in the specified path. The `encoding` parameter is set to `"ISO-8859-1"`, indicating the character encoding of the file. The `header=None` argument specifies that the file does not contain a header row.

3. **Selecting specific columns**: 
   `selected_full_data = full_data.iloc[:, [0, 1, 2, 3, 4, 5]]
    selected_full_data.columns = ['Station Code', 'Station Name', 'Year', 'Month', 'Avg Temp [°C]', 'Status TEMP']`
   This snippet selects certain columns from the loaded data (columns at index positions 0 to 5) and then renames these columns for clarity and ease of understanding.

4. **Filtering data based on a condition**: 
   `filtered_full_data = selected_full_data[selected_full_data['Status TEMP'] != 8]`
   This line filters out rows where the value in the 'Status TEMP' column is "8". The resulting DataFrame, `filtered_full_data`, contains only the rows that meet this condition.



In [33]:
# Required Libraries
import pandas as pd

# Load the entire CSV file and select the specified columns
full_data = pd.read_csv("../TemperatureData/PartialYearsFiles/k_m_t_2019.csv", encoding="ISO-8859-1", header=None)
selected_full_data = full_data.iloc[:, [0, 1, 2, 3, 4, 5]]
selected_full_data.columns = ['Station Code', 'Station Name', 'Year', 'Month', 'Avg Temp [°C]', 'Status TEMP']

# Filter out rows where the 'Status TEMP' column contains the value "8"
filtered_full_data = selected_full_data[selected_full_data['Status TEMP'] != 8]

filtered_full_data

,Station Code,Station Name,Year,Month,Avg Temp [°C],Status TEMP
0,249180010,PSZCZYNA,2019,1,-2.1,NaN
1,249180010,PSZCZYNA,2019,2,2.8,NaN
2,249180010,PSZCZYNA,2019,3,5.9,NaN
3,249180010,PSZCZYNA,2019,4,9.7,NaN
4,249180010,PSZCZYNA,2019,5,11.7,NaN
...,...,...,...,...,...,...
793,254220090,OLECKO,2019,8,18.5,NaN
794,254220090,OLECKO,2019,9,13.1,NaN
795,254220090,OLECKO,2019,10,9.2,NaN
796,254220090,OLECKO,2019,11,4.7,NaN


As we can see data contains all data from all meteorological stations in Poland 

To provide better understanding of provided data, national database of Poland provide us .txt file with description of columns and data in them.
For ease of understanding below is copied and translated content of mentioned .txt file in Markdown format   

#### Description of data

In [34]:
average_temperature_per_month = filtered_full_data.groupby('Month')['Avg Temp [°C]'].mean()

average_temperature_per_month

Month
1     -3.069118
2      1.885075
3      4.737879
4      8.734848
5     11.668182
6     20.843750
7     18.079687
8     18.996923
9     13.167692
10     9.652308
11     5.936508
12     2.212500
Name: Avg Temp [°C], dtype: float64

In [35]:
def append_average_monthly_temperature(existing_df: pd.DataFrame, csv_path: str) -> pd.DataFrame:
    """
    Append the average temperature for each month across all stations from the given CSV file to an existing dataframe.
    
    Parameters:
    - existing_df (pd.DataFrame): The existing dataframe to which the data should be appended.
    - csv_path (str): Path to the CSV file.
    
    Returns:
    - pd.DataFrame: The updated dataframe with the appended data.
    """
    
    # Load the CSV file and select the specified columns
    full_data = pd.read_csv(csv_path, encoding="ISO-8859-1", header=None)
    selected_data = full_data.iloc[:, [0, 1, 2, 3, 4, 5]]
    selected_data.columns = ['Station Code', 'Station Name', 'Year', 'Month', 'Avg Temp [°C]', 'Status TEMP']

    # Filter out rows where the 'Status TEMP' column contains the value "8"
    filtered_data = selected_data[selected_data['Status TEMP'] != 8]

    # Group by 'Year' and 'Month', then calculate the average temperature for each month across all stations
    average_temperature = filtered_data.groupby(['Year', 'Month'])['Avg Temp [°C]'].mean().reset_index()
    
    # Append the calculated data to the existing dataframe
    updated_df = existing_df._append(average_temperature, ignore_index=True)
    
    return updated_df


In [36]:
csv_files = [
    "../TemperatureData/PartialYearsFiles/k_m_t_2019.csv",
    "../TemperatureData/PartialYearsFiles/k_m_t_2020.csv",
    "../TemperatureData/PartialYearsFiles/k_m_t_2021.csv",
    "../TemperatureData/PartialYearsFiles/k_m_t_2022.csv",
    "../TemperatureData/PartialYearsFiles/k_m_t_2023.csv"
]

# Initializing an empty DataFrame to hold the aggregated data
aggregated_df = pd.DataFrame(columns=['Year', 'Month', 'Avg Temp [°C]'])

# Aggregating data from all CSV files
for file in csv_files:
    aggregated_df = append_average_monthly_temperature(aggregated_df,file)

aggregated_df

/tmp/ipykernel_48129/1575111083.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = existing_df._append(average_temperature, ignore_index=True)


,Year,Month,Avg Temp [°C]
0,2019,1,-3.069118
1,2019,2,1.885075
2,2019,3,4.737879
3,2019,4,8.734848
4,2019,5,11.668182
5,2019,6,20.843750
6,2019,7,18.079687
7,2019,8,18.996923
8,2019,9,13.167692
9,2019,10,9.652308


In [37]:
aggregated_df.to_csv("aggregatedDataTemperature.csv",index=False, encoding="ISO-8859-1")